In [28]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import json


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Froms
from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")
# --- 1. Constantes de Negócio ---

# Caminho para seu arquivo de distâncias
ARQUIVO_DISTANCIAS = '../dados/distancias_todas_combinacoes.csv'

# Constantes para a fórmula de Leadtime Teórico
K = 5
p = 6
q = 4
C = 3
alpha = 1

ARQUIVO_JSON_FILIAIS = '../dados/filiais_inferior_30.json'
DATA_ANALISE = '2025-01-16'  # Formato AAAA-MM-DD
TABELA_BIGQUERY = 'planejamento-animale-292719.checklists_rollout.ANIMALE_checklist'
PATH_CREDENCAIS = '../bd/planejamento-animale-292719-296d49ccdea6.json'


In [29]:

try:
    print(f"Carregando filiais do arquivo: {ARQUIVO_JSON_FILIAIS}...")
    
    with open(ARQUIVO_JSON_FILIAIS, 'r', encoding='utf-8') as f:
        dados_filiais = json.load(f)
        
    df_filiais = pd.DataFrame(dados_filiais)
    
    lista_nomes_filiais = df_filiais['FILIAL'].unique().tolist()
    
    if not lista_nomes_filiais:
        print("Atenção: Nenhuma filial foi encontrada no arquivo JSON.")
    else:
        print(f"Sucesso. {len(lista_nomes_filiais)} filiais únicas encontradas.")

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{ARQUIVO_JSON_FILIAIS}'")
    lista_nomes_filiais = []
except Exception as e:
    print(f"Erro ao ler o arquivo JSON: {e}")
    lista_nomes_filiais = []


# --- Montar e Executar a Query ---

if lista_nomes_filiais:
    
    # Ex: Transforma ['Filial A', 'Filial B'] em "('Filial A', 'Filial B')"
    filiais_para_query = "','".join(lista_nomes_filiais)
    filiais_para_query = f"('{filiais_para_query}')"

    query = f"""
        SELECT SKU, FILIAL, VELOCIDADE_VENDA, ALVO, PRESENTE, TRANSITO, 
        EST_TOTAL, CONT_RUPTURA, CONT_FALTA, CONT_EXCESSO, VOLUME_EXCESSO, REGULADOR
        FROM {TABELA_BIGQUERY}
        WHERE FILIAL IN {filiais_para_query}
        AND DATA = '{DATA_ANALISE}'
    """
    try:
        # --- Buscar os dados ---
        
        print("Executando consulta no BigQuery...")
        df_resultado = tableToPandas(query, 'planejamento-animale-292719', cred)
        
        print("\n--- Resultado da Consulta ---")
        if df_resultado.empty:
            print("A consulta não retornou dados.")
        else:
            print(df_resultado)

    except Exception as e:
        print(f"\nErro ao executar a consulta no BigQuery: {e}")

else:
    print("\nAnálise não executada pois nenhuma filial foi carregada.")

Carregando filiais do arquivo: ../dados/filiais_inferior_30.json...
Sucesso. 62 filiais únicas encontradas.
Executando consulta no BigQuery...

--- Resultado da Consulta ---
                           SKU                 FILIAL  VELOCIDADE_VENDA  ALVO  \
0       03.01.1475-09030-TAM_2  ANIMALE ALPHAVILLE CM          0.000000   1.0   
1       03.01.1475-09030-TAM_3  ANIMALE ALPHAVILLE CM          0.000000   1.0   
2        03.01.1481-0023-TAM_2  ANIMALE ALPHAVILLE CM          0.000000   1.0   
3        03.01.1481-0023-TAM_3  ANIMALE ALPHAVILLE CM          0.000000   1.0   
4        03.01.1481-0023-TAM_4  ANIMALE ALPHAVILLE CM          0.000000   1.0   
...                        ...                    ...               ...   ...   
109710   52.57.0005-0005-TAM_3              BUZIOS CM          0.111111   2.0   
109711  52.57.0006-04501-TAM_2              BUZIOS CM          0.142857   2.0   
109712  52.57.0006-04501-TAM_3              BUZIOS CM          0.000000   1.0   
109713   52.N8.0

In [30]:
def carregar_matriz_distancia(caminho_csv):
    """Carrega o CSV de distâncias."""
    print(f"Carregando matriz de distâncias de: {caminho_csv}...")
    try:
        df_dist = pd.read_csv(
            caminho_csv,
            usecols=['Filial_A', 'Filial_B', 'Distancia_km'] 
        )
        print(f"Matriz de distâncias carregada com {len(df_dist)} combinações.")
        return df_dist
    except FileNotFoundError:
        print(f"Erro: Arquivo CSV não encontrado em '{caminho_csv}'")
        return pd.DataFrame(columns=['Filial_A', 'Filial_B', 'Distancia_km'])
    except Exception as e:
        print(f"Erro ao carregar CSV: {e}")
        return pd.DataFrame(columns=['Filial_A', 'Filial_B', 'Distancia_km'])

def calcular_ranking_global_nxn(df_estoque, df_distancias):

    if df_estoque.empty or df_distancias.empty:
        print("Erro: Dados de estoque ou distâncias estão vazios. Encerrando.")
        return pd.DataFrame()

    # --- 1. Preparação dos Dados ---
    df_estoque['FILIAL'] = df_estoque['FILIAL'].astype(str).str.strip()
    df_distancias['Filial_A'] = df_distancias['Filial_A'].astype(str).str.strip()
    df_distancias['Filial_B'] = df_distancias['Filial_B'].astype(str).str.strip()

    if 'PRESENTE' in df_estoque.columns:
        df_estoque = df_estoque.rename(columns={'PRESENTE': 'EST_DISP'})
    
    df_estoque['DEMANDA_LIQUIDA'] = np.maximum(0, df_estoque['ALVO'].fillna(0) - df_estoque['EST_TOTAL'].fillna(0))
    
    # --- 2. Identificar TODAS as Fontes ---
    df_fontes = df_estoque[
        df_estoque['VOLUME_EXCESSO'].fillna(0) > 0
    ][['SKU', 'FILIAL', 'VOLUME_EXCESSO']].rename(columns={'FILIAL': 'FILIAL_FONTE'})

    if df_fontes.empty:
        print("Nenhuma loja-fonte com estoque em excesso foi encontrada.")
        return pd.DataFrame()

    # --- 3. Identificar TODOS os Destinos ---
    df_destinos = df_estoque[
        ( (df_estoque['DEMANDA_LIQUIDA'] > 0) | (df_estoque['CONT_RUPTURA'] == 1) )
    ][[
        'SKU', 'FILIAL', 'DEMANDA_LIQUIDA', 
        'CONT_RUPTURA', 'CONT_FALTA',
        'VELOCIDADE_VENDA'
    ]].rename(columns={'FILIAL': 'FILIAL_DESTINO'})
    
    if df_destinos.empty:
        print("Nenhuma loja-destino com demanda encontrada.")
        return pd.DataFrame()

    # --- 4. Criar Pares de Transferência (N x N) ---
    df_pares = pd.merge(df_fontes, df_destinos, on='SKU')
    df_pares = df_pares[df_pares['FILIAL_FONTE'] != df_pares['FILIAL_DESTINO']]

    if df_pares.empty:
        print("Nenhum par de transferência válido (Fonte != Destino) foi encontrado.")
        return pd.DataFrame()

    # --- 5. Juntar Distâncias ---
    df_ranking = pd.merge(
        df_pares, 
        df_distancias,
        left_on=['FILIAL_FONTE', 'FILIAL_DESTINO'], 
        right_on=['Filial_A', 'Filial_B']
    )
    
    if df_ranking.empty:
        print("Erro: Nenhum par de transferência encontrou uma distância correspondente no CSV.")
        return pd.DataFrame()
    
    # --- 6. Cálculo dos Fatores da Fórmula ---
    
    df_ranking['FATOR_PREFERENCIA'] = np.where(
        df_ranking['FILIAL_FONTE'] == "CENTRO DE DISTRIBUICAO", 
        alpha, 
        1
    )
    
    df_ranking['LEADTIME'] = C * df_ranking['Distancia_km']
    df_ranking['CUSTO_FRETE'] = K * df_ranking['Distancia_km']
    fator_logistica = (1 / (df_ranking['LEADTIME'] + 1)) + (1 / (df_ranking['CUSTO_FRETE'] + 1))

    cont_ruptura = df_ranking['CONT_RUPTURA'].fillna(0)
    cont_falta = df_ranking['CONT_FALTA'].fillna(0)
    fator_demanda = (cont_ruptura * p) + (cont_falta * q)

    velocidade_venda = df_ranking['VELOCIDADE_VENDA'].fillna(0)
    
    # --- 7. Ranking Bruto e Final ---
    df_ranking['RANKING_TRANSFERENCIA_BRUTO'] = (
        velocidade_venda * fator_logistica * fator_demanda * df_ranking['FATOR_PREFERENCIA']
    )
    
    df_ranking['QTD_A_TRANSFERIR'] = np.minimum(
        df_ranking['VOLUME_EXCESSO'], df_ranking['DEMANDA_LIQUIDA']
    )
    
    # Zera o ranking se a QTD a transferir for 0 (sem estoque)
    df_ranking['RANKING_TRANSFERENCIA'] = df_ranking['RANKING_TRANSFERENCIA_BRUTO']
    df_ranking.loc[df_ranking['QTD_A_TRANSFERIR'] == 0, 'RANKING_TRANSFERENCIA'] = 0

    # --- 8. Limpar e Ordenar ---
    df_resultado = df_ranking.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)
    
    colunas_finais = [
        'SKU', 
        'FILIAL_FONTE', 
        'FILIAL_DESTINO', 
        'RANKING_TRANSFERENCIA',
        'VELOCIDADE_VENDA'
        'CONT_FALTA'
        'QTD_A_TRANSFERIR',
        'VOLUME_EXCESSO',
        'DEMANDA_LIQUIDA',
        'Distancia_km',
        'FATOR_PREFERENCIA',
        'LEADTIME', 'CUSTO_FRETE', 'CONT_RUPTURA', 'CONT_FALTA'
    ]
    
    colunas_existentes = [col for col in colunas_finais if col in df_resultado.columns]
    
    print(f"  Ranking global N x N calculado.")
    print(f"  Total de transferências possíveis sugeridas: {df_resultado[df_resultado['RANKING_TRANSFERENCIA'] > 0].shape[0]}")
    
    return df_resultado[colunas_existentes].reset_index(drop=True)

def encontrar_melhor_match_para_sku(df_ranking_global, sku_desejado):
    # Verifica se o DataFrame de entrada está vazio
    if df_ranking_global.empty:
        print("DataFrame de ranking global está vazio.")
        return pd.DataFrame()
        
    # Limpa a string de entrada e a coluna do DataFrame por segurança
    sku_desejado = str(sku_desejado).strip()
    df_sku = df_ranking_global[
        df_ranking_global['SKU'].astype(str).str.strip() == sku_desejado
    ]
    # combinções possíveis:
    
    
    if df_sku.empty:
        return pd.DataFrame(columns=df_ranking_global.columns)

    return df_sku.head(1)

def encontrar_melhor_match_para_todos_skus(df_ranking_global):
    if df_ranking_global.empty:
        print("DataFrame de ranking global está vazio.")
        return pd.DataFrame()

    print(f"\nEncontrando o melhor match para {df_ranking_global['SKU'].nunique()} SKUs únicos...")
    
    df_best_matches = df_ranking_global.groupby('SKU').first()
    
    # O groupby torna 'SKU' o índice. Vamos resetar para ser uma coluna.
    df_best_matches = df_best_matches.reset_index()
    
    # Re-ordenar pela prioridade geral
    df_best_matches = df_best_matches.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)
    
    print(f"Melhores matches encontrados.")
    return df_best_matches

In [31]:
if __name__ == "__main__":
    
    
    print("--- ETAPA 1: Preparando Dados (Lojas + CD) ---")
    

    
    df_distancias_total = carregar_matriz_distancia(ARQUIVO_DISTANCIAS)
    df_estoque_dia = df_resultado.copy()

    # Limpa a coluna FILIAL original
    df_estoque_dia['FILIAL'] = df_estoque_dia['FILIAL'].astype(str).str.strip()

    df_cd_estoque = df_estoque_dia.groupby('SKU', as_index=False)['REGULADOR'].first()
    df_cd_estoque['FILIAL'] = "CENTRO DE DISTRIBUICAO".strip()
    df_cd_estoque = df_cd_estoque.rename(columns={'REGULADOR': 'VOLUME_EXCESSO'})
    
    df_estoque_completo = pd.concat([df_estoque_dia, df_cd_estoque], ignore_index=True)
    
    print("\n--- ETAPA 2: Calculando Ranking Global N-para-N (Motor) ---")

    df_ranking_global = calcular_ranking_global_nxn(
        df_estoque_completo, 
        df_distancias_total,
    )
    
    if not df_ranking_global.empty:
        print("\n--- ETAPA 3: Testando a Função 1 (Match para SKU específico) ---")
        sku = "52.39.0307-0005-TAM_4"
        
        df_melhor_match_sku = encontrar_melhor_match_para_sku(df_ranking_global, sku)
        
        if not df_melhor_match_sku.empty:
            print(f"Melhor match encontrado para {sku}:")
            print(df_melhor_match_sku.to_markdown(index=False, floatfmt=".4f"))
        else:
            print(f"Nenhum match encontrado para {sku}")
        
        print("\n--- ETAPA 4: Testando a Função 2 (Melhor match para TODOS os SKUs) ---")
        df_melhores_matches_todos_skus = encontrar_melhor_match_para_todos_skus(df_ranking_global)
        
        print("Melhores matches encontrados (um por SKU):")
        print(df_melhores_matches_todos_skus.to_markdown(index=False, floatfmt=".4f"))

    else:
        print("\nCálculo do ranking N-para-N não gerou resultados. Funções de match não executadas.")

--- ETAPA 1: Preparando Dados (Lojas + CD) ---
Carregando matriz de distâncias de: ../dados/distancias_todas_combinacoes.csv...
Matriz de distâncias carregada com 1953 combinações.

--- ETAPA 2: Calculando Ranking Global N-para-N (Motor) ---
  Ranking global N x N calculado.
  Total de transferências possíveis sugeridas: 7002

--- ETAPA 3: Testando a Função 1 (Match para SKU específico) ---
Melhor match encontrado para 52.39.0307-0005-TAM_4:
| SKU                   | FILIAL_FONTE          | FILIAL_DESTINO         |   RANKING_TRANSFERENCIA |   VOLUME_EXCESSO |   DEMANDA_LIQUIDA |   Distancia_km |   FATOR_PREFERENCIA |   LEADTIME |   CUSTO_FRETE |   CONT_RUPTURA |   CONT_FALTA |
|:----------------------|:----------------------|:-----------------------|------------------------:|-----------------:|------------------:|---------------:|--------------------:|-----------:|--------------:|---------------:|-------------:|
| 52.39.0307-0005-TAM_4 | ANIMALE BARRA SHOP CM | ANIMALE VILLA LOBOS CM |

In [33]:
def exibir_debug_sku(sku_desejado, df_estoque_completo, df_ranking_global):
    """
    Exibe um relatório de debug detalhado para um único SKU,
    mostrando fontes, destinos e todos os parâmetros de cálculo.
    """
    
    print("="*60)
    print(f"INICIANDO DEBUG PARA O SKU: {sku_desejado}")
    print("="*60)
    
    # --- 1. Limpeza e Validação ---
    sku_desejado = str(sku_desejado).strip()
    
    df_sku_data = df_estoque_completo[
        df_estoque_completo['SKU'].astype(str).str.strip() == sku_desejado
    ].copy()
    
    if df_sku_data.empty:
        print(f"ERRO: SKU '{sku_desejado}' não foi encontrado no DataFrame de estoque.")
        return

    # --- 2. Exibir Fontes (Quem TEM o SKU) ---
    print("\n## 1. FONTES (Quem tem este SKU)")
    df_fontes = df_sku_data[df_sku_data['VOLUME_EXCESSO'].fillna(0) > 0]
    
    if df_fontes.empty:
        print("Nenhuma filial fonte encontrada com excesso/estoque deste SKU.")
    else:
        print(df_fontes[['FILIAL', 'VOLUME_EXCESSO']].to_markdown(index=False))

    # --- 3. Exibir Destinos (Quem PRECISA do SKU) ---
    print("\n## 2. DESTINOS (Quem precisa deste SKU)")
    
    if 'DEMANDA_LIQUIDA' not in df_sku_data.columns:
        print("  (Calculando 'DEMANDA_LIQUIDA' para o debug...)")
        df_sku_data['DEMANDA_LIQUIDA'] = np.maximum(0, df_sku_data['ALVO'].fillna(0) - df_sku_data['EST_TOTAL'].fillna(0))

    df_destinos = df_sku_data[
        (df_sku_data['DEMANDA_LIQUIDA'].fillna(0) > 0) | 
        (df_sku_data['CONT_RUPTURA'].fillna(0) == 1)
    ]
    
    if df_destinos.empty:
        print("Nenhuma filial destino encontrada com demanda para este SKU.")
    else:
        cols_destino = ['FILIAL', 'DEMANDA_LIQUIDA', 'CONT_RUPTURA', 'CONT_FALTA', 'VELOCIDADE_VENDA']
        cols_destino_existentes = [col for col in cols_destino if col in df_destinos.columns]
        print(df_destinos[cols_destino_existentes].to_markdown(index=False, floatfmt=".2f"))

    # --- 4. Exibir Cálculos (O "Matching" N-para-N) ---
    print("\n## 3. CÁLCULO DE RANKING (Todos os Matches Possíveis)")
    
    # Filtra o ranking global (que agora tem 'VELOCIDADE_VENDA')
    df_ranking_sku = df_ranking_global[
        df_ranking_global['SKU'].astype(str).str.strip() == sku_desejado
    ]
    
    if df_ranking_sku.empty:
        print("Nenhum match (Fonte -> Destino) foi encontrado ou calculado para este SKU.")
    else:
        print("Mostrando todos os parâmetros calculados para cada combinação:")
        print(df_ranking_sku.to_markdown(index=False, floatfmt=".4f"))
        
        print("\n---")
        print("MELHOR MATCH ENCONTRADO:")
        print(df_ranking_sku.head(1).to_markdown(index=False, floatfmt=".4f"))
        
    print("="*60)
    print(f"FIM DO DEBUG PARA O SKU: {sku_desejado}")
    print("="*60)
    
    # Salva o arquivo de debug específico do SKU
    try:
        nome_debug_excel = f"debug_sku_{sku_desejado}.xlsx"
        df_ranking_sku.to_excel(nome_debug_excel, index=False)
        print(f"Arquivo de debug salvo em: '{nome_debug_excel}'")
    except Exception as e:
        print(f"Erro ao salvar o arquivo de debug: {e}")
    

    # --- 4. Execução Principal ---
# --- 4. Execução Principal ---
if __name__ == "__main__":
    
    df_estoque_dia = df_resultado.copy()
    df_distancias_total = carregar_matriz_distancia(ARQUIVO_DISTANCIAS)

    df_estoque_dia['FILIAL'] = df_estoque_dia['FILIAL'].astype(str).str.strip()

    if 'REGULADOR' in df_estoque_dia.columns:
        df_cd_estoque = df_estoque_dia.groupby('SKU', as_index=False)['REGULADOR'].first()
        df_cd_estoque['FILIAL'] = "CENTRO DE DISTRIBUICAO".strip()
        df_cd_estoque = df_cd_estoque.rename(columns={'REGULADOR': 'VOLUME_EXCESSO'})
        
        df_estoque_completo = pd.concat([df_estoque_dia, df_cd_estoque], ignore_index=True)
    else:
        print("Aviso: Coluna 'REGULADOR' não encontrada. O CD não será criado como fonte.")
        df_estoque_completo = df_estoque_dia.copy()

    
    df_ranking_global = calcular_ranking_global_nxn(
        df_estoque_completo, 
        df_distancias_total,
    )
    
    if not df_ranking_global.empty:
        
        exibir_debug_sku(
            sku_desejado="52.39.0307-0005-TAM_4", # <-- O SKU que você escolheu
            df_estoque_completo=df_estoque_completo,
            df_ranking_global=df_ranking_global
        )
        
        df_melhores_matches = encontrar_melhor_match_para_todos_skus(df_ranking_global)
        nome_arquivo = "ranking_transferencias.xlsx"
        
        try:
            with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
                df_ranking_global.to_excel(
                    writer, 
                    sheet_name='Ranking_Completo_NxN', 
                    index=False
                )
                
                df_melhores_matches.to_excel(
                    writer, 
                    sheet_name='Melhor_Match_por_SKU', 
                    index=False
                )
            
            print(f"Sucesso! Resultados GLOBAIS salvos em '{nome_arquivo}' com 2 abas.")

        except Exception as e:
            print(f"Erro ao salvar o arquivo Excel global: {e}")

    else:
        print("\nCálculo do ranking N-para-N não gerou resultados. Nada foi salvo.")

Carregando matriz de distâncias de: ../dados/distancias_todas_combinacoes.csv...
Matriz de distâncias carregada com 1953 combinações.
  Ranking global N x N calculado.
  Total de transferências possíveis sugeridas: 7002
INICIANDO DEBUG PARA O SKU: 52.39.0307-0005-TAM_4

## 1. FONTES (Quem tem este SKU)
| FILIAL                |   VOLUME_EXCESSO |
|:----------------------|-----------------:|
| ANIMALE BARRA SHOP CM |                2 |

## 2. DESTINOS (Quem precisa deste SKU)
  (Calculando 'DEMANDA_LIQUIDA' para o debug...)
| FILIAL                  |   DEMANDA_LIQUIDA |   CONT_RUPTURA |   CONT_FALTA |   VELOCIDADE_VENDA |
|:------------------------|------------------:|---------------:|-------------:|-------------------:|
| ANIMALE ALPHAVILLE CM   |              1.00 |              1 |            0 |               0.00 |
| ANIMALE GAL CAMPINAS CM |              1.00 |              1 |            0 |               0.00 |
| ANIMALE HIGIENOPOLIS CM |              1.00 |              1 |   